# Load libraries and functions

In [51]:
with open('libraries.py') as f:
    code = f.read()
exec(code)

with open('functions/functions.py') as f:
    code = f.read()
exec(code)

### Read DAG creation functions

In [52]:
with open('functions/manual_DAG_function.py') as f:
    code = f.read()
exec(code)

with open('functions/naive_DAG_function.py') as f:
    code = f.read()
exec(code)

with open('functions/firstLastTask_DAG_function.py') as f:
    code = f.read()
exec(code)

with open('functions/partitioned_DAG_function.py') as f:
    code = f.read()
exec(code)

with open('functions/condition_DAG_function.py') as f:
    code = f.read()
exec(code)

In [53]:
def get_tasks(onet_data_path,
              occupation_code):

    # Load the data
    onet = pd.read_csv(onet_data_path)
    onet = onet.sort_values(by=['year', 'occ_code', 'occ_title', 'task_id'])
    onet = onet[onet['year'] == 2023].reset_index(drop=True)

    # Get list of tasks
    my_df = onet[(onet.occ_code == f'{occupation_code}') & (onet.year == 2023)]
    tasks = my_df['task'].unique().tolist()
    return tasks

## Main Code

In [54]:
# determine user
user = getpass.getuser()
if user == 'peymansh':
    main_folder_path = '/Users/peymansh/Dropbox (MIT)/Research/AI and Occupations/ai-exposure'
    data_path = f'{main_folder_path}/output'

### Run DAGs

In [55]:
onet_data_path = f'{data_path}/data/onet_occupations_yearly.csv'

# list of occupations to create DAGs for
occupation_list = ['travelAgents', 'insuranceUnderwriters', 'pileDriverOperators', 
                   'dredgeOperators', 'gradersAndSortersForAgriculturalProducts', 'reinforcingIronAndRebarWorkers',
                   'insuranceAppraisersForAutoDamage', 'floorSandersAndFinishers', 'dataEntryKeyer', 
                   'athletesAndSportsCompetitors', 'audiovisualEquipmentInstallerAndRepairers', 'hearingAidSpecialists', 
                   'personalCareAides', 'proofreadersAndCopyMarkers', 'chiropractors', 
                   'shippingReceivingAndInventoryClerks', 'cooksShortOrder', 'orthodontists',
                   'subwayAndStreetcarOperators', 'packersAndPackagersHand', 'hoistAndWinchOperators', 
                   'forgingMachineSettersOperatorsAndTenders', 'avionicsTechnicians', 'dishwashers', 
                   'dispatchersExceptPoliceFireAndAmbulance', 'familyMedicinePhysicians', 'MachineFeedersAndOffbearers'
                   ]

occupation_list = ['packersAndPackagersHand', 'hoistAndWinchOperators', 
                   'forgingMachineSettersOperatorsAndTenders', 'dishwashers', 
                   'dispatchersExceptPoliceFireAndAmbulance', 'familyMedicinePhysicians', 'MachineFeedersAndOffbearers'
                   ]

occupation_list = ['dishwashers', 
                   'dispatchersExceptPoliceFireAndAmbulance', 'familyMedicinePhysicians', 'MachineFeedersAndOffbearers'
                   ]

In [56]:
for occupation in occupation_list:
    print(f'--------------- Running: {occupation} ---------------')

    # Generate occupation-specific strings
    GPT_input_occupation, plot_title_occupation, occupation_code, occupation_folder = pick_occupation(occupation)

    # Get occupation tasks
    tasks = get_tasks(onet_data_path, occupation_code)
    tasks = [task.replace("'", "") for task in tasks] # remove apastrophes so partitions DAG code can handle task strings
    print(f'Number of tasks: {len(tasks)}')


    # 1) Manual DAG
    if occupation in ['travelAgents', 'insuranceUnderwriters', 'pileDriverOperators']:
        create_manual_DAG(occupation,
                          input_filename = f'{occupation_folder}/{occupation}_AM.csv',
                          output_filename = f'{occupation_folder}/{occupation}_M_DAG_df.csv')
        
    # 2) Naive DAG(s)
    print('\n*** Naive DAG ***')
    naive_DAG(GPT_input_occupation,
              tasks,
              lastTask_output_filename = f'{occupation_folder}/{occupation}_N_lastTasks.csv',
              output_DAG_filename_naive = f'{occupation_folder}/{occupation}_N_GPT_DAG_df.csv',
              output_DAG_filename_naiveTwoStep = f'{occupation_folder}/{occupation}_N2_GPT_DAG_df.csv',
              conditioned_DAG_output_filename = f'{occupation_folder}/{occupation}_CN_GPT_DAG_df.csv')
    
    # 3) First-last-task DAG
    print('\n*** First-last-task DAG ***')
    firstLastTask_DAG(GPT_input_occupation,
                      tasks,
                      firstLastTask_output_filename = f'{occupation_folder}/{occupation}_FLT_tasks.csv',
                      firstLastTask_DAG_output_filename = f'{occupation_folder}/{occupation}_FLT_GPT_DAG_df.csv',
                      conditioned_DAG_output_filename = f'{occupation_folder}/{occupation}_CFLT_GPT_DAG_df.csv')
    
    # 4) Partitioned DAG
    print('\n*** Partitioned DAG ***')
    partitioned_DAG(GPT_input_occupation,
                    tasks,
                    lastTask_output_filename = f'{occupation_folder}/{occupation}_P_lastTasks.csv',
                    partitions_output_filename = f'{occupation_folder}/{occupation}_P_partitions.csv',
                    partitioned_DAG_output_filename = f'{occupation_folder}/{occupation}_P_GPT_DAG_df.csv',
                    conditioned_partitioned_DAG_output_filename = f'{occupation_folder}/{occupation}_CP_GPT_DAG_df.csv')
    
    print('')

--------------- Running: dishwashers ---------------
Number of tasks: 13

*** Naive DAG ***
Count of triangles: 5
Number of quadrilaterals: 0
Number of AC-DC edges to remove: 0
Number of AC edges to remove: 5
Total number of edges to remove: 5

*** First-last-task DAG ***
Count of triangles: 2
Number of quadrilaterals: 0
Number of AC-DC edges to remove: 0
Number of AC edges to remove: 2
Total number of edges to remove: 2

*** Partitioned DAG ***
Count of triangles: 14
Number of quadrilaterals: 0
Number of AC-DC edges to remove: 0
Number of AC edges to remove: 13
Total number of edges to remove: 13

--------------- Running: dispatchersExceptPoliceFireAndAmbulance ---------------
Number of tasks: 13

*** Naive DAG ***
Count of triangles: 110
Number of quadrilaterals: 6
Number of AC-DC edges to remove: 0
Number of AC edges to remove: 80
Total number of edges to remove: 80

*** First-last-task DAG ***
Count of triangles: 90
Number of quadrilaterals: 0
Number of AC-DC edges to remove: 0
Num